In [1]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
import polars as pl
import random
from dotenv import load_dotenv
load_dotenv()
import utils
%load_ext autoreload
%autoreload 2

In [2]:
model = AzureChatOpenAI(
    deployment_name= "gpt-4o",
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version="2023-09-15-preview",
    azure_endpoint="https://acsstscdamoai02.openai.azure.com/"
)

C:\Users\monojit.mandal\AppData\Local\Temp\ipykernel_13404\2458642598.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  model = AzureChatOpenAI(


In [3]:
df_offers = pl.read_csv('data/sample_offers.csv')
df_offers.head()

offer,price_per_unit,quantity,bundling_unit,bundling_amount,bundling_discount,payment_term,payment_term_markup,delivery_timeline,contract_period,contract_inflation,rebates_threshold_unit,rebates_discount,warranty,incoterms
str,i64,i64,str,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,str
"""Price per unit: $95 Quantity: …",95,10000,null,20000,5,"""NET60""",3,10,2,3,12000,2,1,"""DDP"""
"""Price per unit: $100 Quantity:…",100,10000,null,null,null,"""NET10""",0,7,1,0,11000,1,1,"""FOB"""
"""I want to offer product for 13…",130,null,null,null,null,null,null,null,null,null,null,null,null,null


#### Logic Flows
* Extract levers from negotiation chat
* If there are any lever values present, negotiate and generate revised offer based on revised TCO
* If not then check if there are any lever mentioned in chat on which Supplier wants to negotiate
* If yes, then generate revised offer based on those levers.
* If no, then check if supplier is interested in negotiation.
* If yes then ask for levers which supplier want to negotiate
* If no then check if Supplier agreed with offer.
* If yes no then end conversation

In [4]:
utils.extract_levers_from_text(
    offer_text='quantity:300 with inflation 5% for 2 years',
    model=model,
    example_data=df_offers
)

{'price_per_unit': None,
 'quantity': 300,
 'bundling_unit': None,
 'bundling_amount': None,
 'bundling_discount': None,
 'payment_term': None,
 'payment_term_markup': None,
 'delivery_timeline': None,
 'contract_period': 2,
 'contract_inflation': 5,
 'rebates_threshold_unit': None,
 'rebates_discount': None,
 'warranty': None,
 'incoterms': None}

#### Extract Lever(s) Supplier Wants to Negotiate

In [10]:
utils.understand_levers_to_negotiate(
    offer_text='I think price is too low',
    model = model
)

['price_per_unit']

#### If Supplier Wants to Negotiate But Did not Mention Any Lever: Generate Offer Revising Random Lever Variable

In [6]:
import random
from scipy.optimize import minimize_scalar

offer = utils.ContractOffer(
    price_per_unit = 100,
    quantity = 10000,
    bundling_unit = None,
    bundling_amount = None,
    bundling_discount = None,
    payment_term=utils.Payment(
        term=utils.PaymentTerm.NET10,
        markup=0
    ),
    delivery_timeline = 7,
    contract_period = 1,
    contract_inflation = 0,
    rebates_threshold_unit = 11000,
    rebates_discount = 1,
    warranty = 1,
    incoterms = utils.Incoterms.FOB
)
actual = utils.ContractActual(
    price_per_unit=90,
    quantity = 12000,
    payment_term=utils.PaymentTerm.NET30,
    incoterms=utils.Incoterms.FOB
)
print(f'Existing TCO: {actual.calculate_TCO_yearly(offer=offer)}')
utils.generate_offer_given_TCO_target(
    actual = actual,
    offer = offer,
    TCO_target = 1183000,
    lever_priority = {
        'quantity': 0.4,
        'price_per_unit': 0.4,
        'payment_term': 0.1,
        'incoterm':0.1
    },
    min_lever = {
        'quantity': 10000,
        'price_per_unit': 90
    },
    max_lever = {
        'quantity': 15000,
        'price_per_unit': 150
    }
)


Existing TCO: 1082442.410958904
Randonmly selected lever: incoterm


ContractActual(price_per_unit=90, quantity=12000, bundling_ind=None, payment_term=<PaymentTerm.NET30: 30>, delivery_timeline=None, contract_period=None, warranty=None, incoterms=<Incoterms.EXW: 21000>)

#### If Supplier Wants to Negotiate Lever(s): Generate Offer Accordingly

In [7]:
actual = utils.generate_offer_tuning_levers(
    offer = offer,
    actual=actual,
    tco_hike_pct=10,
    levers=['quantity','price_per_unit'],
    min_lever={
        'quantity': 10000,
        'price_per_unit': 90
    },
    max_lever={
        'quantity': 15000,
        'price_per_unit': 110
    }
)

Randonmly selected lever: quantity
Randonmly selected lever: price_per_unit


#### If Supplier Chat Was Neutral: Ask Which lever Supplier Wants to Negotiate

In [8]:
# negotiation_text = '''I don't think we will be able to make deal'''
negotiation_text = '''I think offer is too low'''
utils.is_interested_to_negotiate(
    offer_text = negotiation_text,
    model = model
)

True

#### If Supplier Does Not Want to Negotiate, End Negotiation Chat

In [11]:
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.messages import HumanMessage,AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

trimmer = trim_messages(
    max_tokens=10000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                '''You are an experienced Prefessional. Your job is to 
                negotiate with Suppliers. Always be polite and keep 
                professional tone throught the conversation''',
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [43]:
# Sample dictionary
data = {'a': None, 'b': None, 'c': None}

# Check if all values in the dictionary are None
all_null = all(value is None for value in data.values())

print("All values are None:", all_null)

All values are None: True


In [48]:
LEVER_PRIORITY = {
    'quantity': 0.4,
    'price_per_unit': 0.4,
    'payment_term': 0.1,
    'incoterm':0.1
}

In [58]:
# Extract levers from negotiation chat
# If there are any lever values present, negotiate and generate revised offer based on revised TCO
# If not then check if there are any lever mentioned in chat on which Supplier wants to negotiate
# If yes, then generate revised offer based on those levers.
# If no, then check if supplier is interested in negotiation.
# If yes then ask for levers which supplier want to negotiate
# If no then end the conversation
base_offer = '''
Price per unit: $95
Quantity: 10,000 units
Bundling: If ordered with additional safety gear ($20,000 total), a 5% discount on the total contract.
Payment Terms: NET30 with a 3% markup if NET60 is requested.
Delivery Timelines: 10 days.
Contract Period Length: 2 years with a 3% price increase in the second year.
Rebates: 2% rebate if the purchase volume exceeds 12,000 units in the first year.
Warranties: 1-year standard warranty.
Incoterms: DDP (Delivered Duty Paid)
'''
supplier_msg = '''I think the price and quantity is too low'''


base_offer = utils.ContractOffer().load_from_data(
    utils.extract_levers_from_text(
        offer_text=base_offer,
        model= model,
        example_data=df_offers
    )
)
print('Existing offer: ',base_offer)
actual = utils.ContractActual(
    price_per_unit=base_offer.price_per_unit,
    quantity=base_offer.quantity,
    payment_term=base_offer.payment_term.term,
    incoterms=base_offer.incoterms
)
tco = actual.calculate_TCO_yearly(base_offer)
print('Existing TCO: ',tco)

levers = utils.extract_levers_from_text(
    offer_text = supplier_msg,
    model = model,
    example_data = df_offers
)
if all(value is None for value in levers.values()) == False:
    supplier_offer = actual.update_from_data(levers)
    print('Supplier offer: ',supplier_offer)

    tco_supplier = supplier_offer.calculate_TCO_yearly(base_offer)
    print('TCO based on supplier offer: ',tco_supplier)

    tco_new = tco*1.1
    levers_to_negotiate = [lever for lever in levers if levers[lever] != None]
    print('Negotiation lever: ',levers_to_negotiate)

    revised_offer = utils.generate_offer_tuning_levers(
        offer=base_offer,
        actual=actual,
        tco_hike_pct=10,
        levers=levers_to_negotiate,
        min_lever={'price_per_unit':actual.price_per_unit,'quantity':actual.quantity},
        max_lever={'price_per_unit':actual.price_per_unit*1.5,'quantity':actual.quantity*1.5}
    )
    print('Revised offer: ',revised_offer)

    if tco_new < tco_supplier:
        print('Share revised offer to supplier')
    else:
        print('Accept supplier offer')
else:
    levers = utils.understand_levers_to_negotiate(
        offer_text=supplier_msg,
        model = model
    )
    if len(levers) != 0:
        print('Levers to negotiate: ',levers)
        revised_offer = actual = utils.generate_offer_tuning_levers(
            offer=base_offer,
            actual=actual,
            tco_hike_pct=10,
            levers=levers,
            min_lever={'price_per_unit':actual.price_per_unit,'quantity':actual.quantity},
            max_lever={'price_per_unit':actual.price_per_unit*1.5,'quantity':actual.quantity*1.5}
        )
        print('Revised offer: ',revised_offer)
    else:
        is_interested = utils.is_interested_to_negotiate(
            offer_text=supplier_msg,
            model = model
        )
        if is_interested:
            print('Supplier is interested to negotiate')
            revised_offer = utils.generate_offer_given_TCO_target(
                offer=base_offer,
                actual=actual,
                TCO_target=tco*1.1,
                lever_priority=LEVER_PRIORITY,
                min_lever={'price_per_unit':actual.price_per_unit,'quantity':actual.quantity},
                max_lever={'price_per_unit':actual.price_per_unit*1.5,'quantity':actual.quantity*1.5}
            )
            print('Revised Offer: ',revised_offer)
        else:
            is_accepted = utils.is_offer_accepted(
                offer_text=supplier_msg,
                model = model
            )
            if is_accepted:
                print('Offer Accepted')
            else:
                print('Supplier does not want to negotiate')

Existing offer:  ContractOffer(price_per_unit=95, quantity=10000, bundling_unit=None, bundling_amount=20000, bundling_discount=5, payment_term=Payment(term=<PaymentTerm.NET60: 60>, markup=0), delivery_timeline=10, contract_period=2, contract_inflation_by_year=[0, 3], rebates_threshold_unit=12000, rebates_discount=2, warranty=1, incoterms=<Incoterms.DDP: 0>)
Existing TCO:  926097.401369863
Supplier does not want to negotiate


ContractOffer(price_per_unit=100, quantity=10000, bundling_unit=None, bundling_amount=None, bundling_discount=None, payment_term=Payment(term=<PaymentTerm.NET10: 10>, markup=0), delivery_timeline=7, contract_period=1, contract_inflation_by_year=[0], rebates_threshold_unit=11000, rebates_discount=1, warranty=1, incoterms=<Incoterms.FOB: 15000>)

In [31]:
revised_offer.calculate_TCO_yearly(base_offer)

1379598.7061643838

#### Steps:
1. We will show 3 equivalent offers to Supplier using UI
2. Supplier will either pick any offer or press negotiate button
3. We will share a form to be filled by supplier
4. We will either accept or generate three counter offers
5. Supplier wants to negotiate further
6. We will ask about priority levers
7. We will generate three equivalent offers based on priority levers
8. Supplier did not chose and negotiate further
9. We will ask Supplier to share final offer in free text.
10. If it is acceptable we will accept it or create a ticket for manual negotiation 

In [25]:
def generate_offer(
    base_offer,
    min_levers,
    max_levers,
    step,
    TCO_hike_threshold_pct,
    supplier_offer = None
):
    MIN_HIKE = 6
    MAX_HIKE = 8
    actual = utils.ContractActual(
        quantity = base_offer.quantity,
        price_per_unit=base_offer.price_per_unit,
        payment_term=base_offer.payment_term.term,
        delivery_timeline= base_offer.delivery_timeline,
        contract_period=base_offer.contract_period,
        warranty=base_offer.warranty,
        incoterms = base_offer.incoterms
    )
    TCO = actual.calculate_TCO_yearly(offer=base_offer)
    match step:
        case 1:
            # generate three offers
            hike = min(TCO_hike_threshold_pct,random.randint(MIN_HIKE,MAX_HIKE))
            print('TCO after 1st hike: ',TCO*(1+hike/100))
            offers = utils.generate_three_eqv_offers(
                offer = base_offer,
                actual = actual,
                tco_hike_pct = hike,
                min_levers = min_levers,
                max_levers = max_levers
            )
            return {'offers':offers,'status':'live'}
        case 2:
            # Evaluate supplier's offer
            tco_supplier = utils.ContractActual().calculate_TCO_yearly(
                offer=supplier_offer
            )
            hike = min(TCO_hike_threshold_pct,2*random.randint(MIN_HIKE,MAX_HIKE))
            print('TCO after 2nd hike: ',TCO*(1+hike/100))
            # If supplier offer is less than hiked TCO then return supplier offer & accept
            if TCO*(1+hike/100) > tco_supplier:
                return {'offers':[supplier_offer],'status':'accepted'}
            else:
                offers = utils.generate_three_eqv_offers(
                    offer = base_offer,
                    actual = actual,
                    tco_hike_pct = hike,
                    min_levers = min_levers,
                    max_levers = max_levers
                )
                return {'offers':offers,'status':'live'}
        case 3:
            hike = min(TCO_hike_threshold_pct,3*random.randint(MIN_HIKE,MAX_HIKE))
            offers = utils.generate_three_eqv_offers(
                offer = base_offer,
                actual = actual,
                tco_hike_pct = hike,
                min_levers = min_levers,
                max_levers = max_levers
            )
            return {'offers':offers,'status':'live'}
        case 4:
            # Evaluate supplier's final offer
            tco_supplier = utils.ContractActual().calculate_TCO_yearly(
                offer=supplier_offer
            )
            hike = min(TCO_hike_threshold_pct,4*random.randint(MIN_HIKE,MAX_HIKE))
            # If our final offer is less than supplier's final offer then we will accept
            if TCO*(1+hike/100) > tco_supplier:
                return {'offers':[supplier_offer],'status':'accepted'}
            # Else we will share our final offer
            else:
                offers = utils.generate_three_eqv_offers(
                    offer = base_offer,
                    actual = actual,
                    tco_hike_pct = hike,
                    min_levers = min_levers,
                    max_levers = max_levers
                )
                return {'offers':offers,'status':'live'}

In [31]:
# Logic to generate three equivalent offers: Price, Quantity, Price and Quantity
base_offer = utils.ContractOffer(
    price_per_unit = 95,
    quantity = 10000,
    bundling_unit = None,
    bundling_amount = 20000,
    bundling_discount = 5,
    payment_term=utils.Payment(
        term=utils.PaymentTerm.NET60,
        markup=3
    ),
    delivery_timeline = 10,
    contract_period = 2,
    contract_inflation = 3,
    rebates_threshold_unit = 12000,
    rebates_discount = 5,
    warranty = 1,
    incoterms = utils.Incoterms.DDP
)
actual = utils.ContractActual(
    quantity = base_offer.quantity,
    price_per_unit=base_offer.price_per_unit,
    payment_term=base_offer.payment_term.term,
    delivery_timeline= base_offer.delivery_timeline,
    contract_period=base_offer.contract_period,
    warranty=base_offer.warranty,
    incoterms = base_offer.incoterms
)
MIN_LEVERS = {'price_per_unit':95,'quantity':10000}
MAX_LEVERS = {'price_per_unit':140,'quantity':14000}
step = 0
response = generate_offer(
    base_offer = base_offer,
    min_levers = MIN_LEVERS,
    max_levers = MAX_LEVERS,
    step = step,
    TCO_hike_threshold_pct = 40,
    supplier_offer = None
)
supplier_response = int(input('Please select offer: 1 or 2 or 3 or 0 to negotiate'))
if supplier_response in [1,2,3]:
    print(
        'Thanks for selecting the offer. Please find the offer details below: \n',
        response['offers'][supplier_response - 1]
    )
else:
    step = step+1
    print('Please provide your offer in the given form')
    offer_data = pl.read_csv('data/supplier_offer_form.csv').rows(named = True)[0]
    print(offer_data)
    supplier_offer = utils.ContractOffer().load_from_data(data = offer_data)
    print('Thanks for sharing your offer: ',supplier_offer)
    response = generate_offer(
        base_offer = base_offer,
        min_levers = MIN_LEVERS,
        max_levers = MAX_LEVERS,
        step = step,
        TCO_hike_threshold_pct = 40,
        supplier_offer = supplier_offer
    )
    if response['status'] == 'accepted':
        print(f"We have accepted your offer: {response['offers'][0]}")
    else:
        supplier_response = int(input('Please select offer: 1 or 2 or 3 or 0 to negotiate'))
        if supplier_response in [1,2,3]:
            print(
                'Thanks for selecting the offer. Please find the offer details below: \n',
                response['offers'][supplier_response - 1]
            )
        else:
            step = step+1
            print(
                '''
                Please share your priority for the lever mentioned below:
                1. Unit Price
                2. Quantity
                3. Incoterms
                4. Payment Terms
                ''')
            print('Thanks for sharing your priority')
            response = generate_offer(
                base_offer = base_offer,
                min_levers = MIN_LEVERS,
                max_levers = MAX_LEVERS,
                step = step,
                TCO_hike_threshold_pct = 40,
                supplier_offer = supplier_offer
            )
            supplier_response = int(input('Please select offer: 1 or 2 or 3 or 0 to negotiate'))
            if supplier_response in [1,2,3]:
                print(
                    'Thanks for selecting the offer. Please find the offer details below: \n',
                    response['offers'][supplier_response - 1]
                )
            else:
                step = step + 1
                final_offer_supplier = input('Please share your final offer:')
                supplier_response = utils.extract_levers_from_text(
                    offer_text=final_offer_supplier,
                    model=model,
                    example_data=df_offers
                )
                supplier_offer = supplier_offer.load_from_data(supplier_response)
                response = generate_offer(
                    base_offer = base_offer,
                    min_levers = MIN_LEVERS,
                    max_levers = MAX_LEVERS,
                    step = step,
                    TCO_hike_threshold_pct = 40,
                    supplier_offer = supplier_offer
                )
                if response['status'] == 'accepted':
                    print(
                        'Thanks for sharing your offer. We are pleased to inform that we accepted your offer'
                    )
                else:
                    print(f'''
                        Thanks for sharing your final offer. 
                        We are raising a ticket {random.randint(111111,555555)}.
                        One of our agent will reach out to you soon'''
                    )

TCO after 1st hike:  1021185.1055000001
Randonmly selected lever: price_per_unit
Offer 1:  ContractActual(price_per_unit=100.81999981291744, quantity=10000, bundling_ind=None, payment_term=<PaymentTerm.NET60: 60>, delivery_timeline=None, contract_period=None, warranty=None, incoterms=<Incoterms.DDP: 0>)
Randonmly selected lever: quantity
Offer 2:  ContractActual(price_per_unit=95, quantity=11021.052543939388, bundling_ind=None, payment_term=<PaymentTerm.NET60: 60>, delivery_timeline=None, contract_period=None, warranty=None, incoterms=<Incoterms.DDP: 0>)
Randonmly selected lever: price_per_unit
Randonmly selected lever: quantity
Offer 3:  ContractActual(price_per_unit=99.84999973695257, quantity=10485.728625165782, bundling_ind=None, payment_term=<PaymentTerm.NET60: 60>, delivery_timeline=None, contract_period=None, warranty=None, incoterms=<Incoterms.DDP: 0>)
Please provide your offer in the given form
{'price_per_unit': 135, 'quantity': 13000, 'bundling_unit': None, 'bundling_amount'

In [ ]:
# Logic to generate three equivalent offers: Price, Quantity, Price and Quantity
base_offer = utils.ContractOffer(
    price_per_unit = 95,
    quantity = 10000,
    bundling_unit = None,
    bundling_amount = 20000,
    bundling_discount = 5,
    payment_term=utils.Payment(
        term=utils.PaymentTerm.NET60,
        markup=3
    ),
    delivery_timeline = 10,
    contract_period = 2,
    contract_inflation = 3,
    rebates_threshold_unit = 12000,
    rebates_discount = 5,
    warranty = 1,
    incoterms = utils.Incoterms.DDP
)
actual = utils.ContractActual(
    quantity = base_offer.price_per_unit,
    price_per_unit=base_offer.quantity,
    payment_term=base_offer.payment_term.term,
    delivery_timeline= base_offer.delivery_timeline,
    contract_period=base_offer.contract_period,
    warranty=base_offer.warranty,
    incoterms = base_offer.incoterms
)
MIN_LEVERS = {'price_per_unit':95,'quantity':10000}
MAX_LEVERS = {'price_per_unit':140,'quantity':14000}
MIN_HIKE = 8
MAX_HIKE = 10
tco_orig = actual.calculate_TCO_yearly(offer=base_offer)
tco_current = tco_orig
hike = random.randint(MIN_HIKE,MAX_HIKE)
tco_current = tco_current*(1+hike/100)
print(f'Offers generating for TCO: {tco_current}')
offers = utils.generate_three_eqv_offers(
    offer=base_offer,
    actual=actual,
    tco_hike_pct=hike,
    min_levers=MIN_LEVERS,
    max_levers=MAX_LEVERS
)
supplier_response = input('Please select offer: 1 or 2 or 3 or 0 to negotiate')
if supplier_response in [1,2,3]:
    print(
        'Thanks for selecting the offer. Please find the offer details below: \n',
        offers[supplier_response - 1]
    )
else:
    print('Please provide your offer in the given form')
    offer_data = pl.read_csv('supplier_offer_form.csv').rows(named = True)[0]
    print(offer_data)
    supplier_offer = utils.ContractOffer().load_from_data(data = offer_data)
    print('Thanks for sharing your offer: ',supplier_offer)
    tco_supplier = utils.ContractActual().calculate_TCO_yearly(offer=supplier_offer)
    hike = random.randint(MIN_HIKE,MAX_HIKE)
    tco_current = tco_current*(1+hike/100)
    if tco_supplier <= tco_current:
        print('Thanks for sharing your offer. We are happy to accept the offer you shared')
    else:
        offers = utils.generate_three_eqv_offers(
            offer=base_offer,
            actual=actual,
            tco_hike_pct=(tco_current - tco_orig)*100/tco_orig,
            min_levers=MIN_LEVERS,
            max_levers=MAX_LEVERS
        )
        supplier_response = input('Please select offer: 1 or 2 or 3 or 0 to negotiate')
        if supplier_response in [1,2,3]:
            print(
                'Thanks for selecting the offer. Please find the offer details below: \n',
                offers[supplier_response - 1]
            )
        else:
            print(
                '''
                Please share your priority for the lever mentioned below:
                1. Unit Price
                2. Quantity
                3. Incoterms
                4. Payment Terms
                ''')
            print('Thanks for sharing your priority')
            hike = random.randint(MIN_HIKE,MAX_HIKE)
            tco_current = tco_current*(1+hike/100)
            print(f'Offers generating for TCO: {tco_current}')
            offers = utils.generate_three_eqv_offers(
                offer=base_offer,
                actual=actual,
                tco_hike_pct=(tco_current - tco_orig)*100/tco_orig,
                min_levers=MIN_LEVERS,
                max_levers=MAX_LEVERS
            )
            supplier_response = input('Please select offer: 1 or 2 or 3 or 0 to negotiate')
            if supplier_response in [1,2,3]:
                print(
                    'Thanks for selecting the offer. Please find the offer details below: \n',
                    offers[supplier_response - 1]
                )
            else:
                final_offer_supplier = input('Please share your final offer:')
                supplier_response = utils.extract_levers_from_text(
                    offer_text=final_offer_supplier,
                    model=model,
                    example_data=df_offers
                )
                supplier_offer = supplier_offer.load_from_data(supplier_response)
                hike = random.randint(MIN_HIKE,MAX_HIKE)
                tco_current = tco_current*(1+hike/100)
                tco_supplier = utils.ContractActual().calculate_TCO_yearly(offer = supplier_offer)
                if tco_supplier <= tco_current:
                    print(
                        'Thanks for sharing your offer. We are pleased to inform that we accepted your offer'
                    )
                else:
                    print(f'''
                        Thanks for sharing your final offer. 
                        We are raising a ticket {random.randint(111111,555555)}.
                        One of our agent will reach out to you soon'''
                    )

In [22]:
# Logic to generate three equivalent offers: Price, Quantity, Price and Quantity
base_offer = utils.ContractOffer(
    price_per_unit = 95,
    quantity = 10000,
    bundling_unit = None,
    bundling_amount = 20000,
    bundling_discount = 5,
    payment_term=utils.Payment(
        term=utils.PaymentTerm.NET60,
        markup=3
    ),
    delivery_timeline = 10,
    contract_period = 2,
    contract_inflation_by_year = [0,3],
    rebates_threshold_unit = 12000,
    rebates_discount = 5,
    warranty = 1,
    incoterms = utils.Incoterms.DDP
)
actual = utils.ContractActual(
    quantity = 10000,
    price_per_unit=95,
    payment_term=utils.PaymentTerm.NET60,
    incoterms = utils.Incoterms.DDP
)
MIN_LEVERS = {'price_per_unit':95,'quantity':10000}
MAX_LEVERS = {'price_per_unit':140,'quantity':14000}
print('Existing TCO: ',actual.calculate_TCO_yearly(offer=base_offer))
utils.generate_three_eqv_offers(
    offer=base_offer,
    actual=actual,
    tco_hike_pct=10,
    min_lever=MIN_LEVERS,
    max_lever=MAX_LEVERS
)

Existing TCO:  935322.5
Randonmly selected lever: price_per_unit
Offer 1:  ContractActual(price_per_unit=104.6999993488467, quantity=10000, bundling_ind=None, payment_term=<PaymentTerm.NET60: 60>, delivery_timeline=None, contract_period=None, warranty=None, incoterms=<Incoterms.DDP: 0>)
Randonmly selected lever: quantity
Offer 2:  ContractActual(price_per_unit=95, quantity=11021.052574398163, bundling_ind=None, payment_term=<PaymentTerm.NET60: 60>, delivery_timeline=None, contract_period=None, warranty=None, incoterms=<Incoterms.DDP: 0>)
Randonmly selected lever: price_per_unit
Randonmly selected lever: quantity
Offer 3:  ContractActual(price_per_unit=99.84999973695257, quantity=10485.728633000672, bundling_ind=None, payment_term=<PaymentTerm.NET60: 60>, delivery_timeline=None, contract_period=None, warranty=None, incoterms=<Incoterms.DDP: 0>)


(ContractActual(price_per_unit=104.6999993488467, quantity=10000, bundling_ind=None, payment_term=<PaymentTerm.NET60: 60>, delivery_timeline=None, contract_period=None, warranty=None, incoterms=<Incoterms.DDP: 0>),
 ContractActual(price_per_unit=95, quantity=11021.052574398163, bundling_ind=None, payment_term=<PaymentTerm.NET60: 60>, delivery_timeline=None, contract_period=None, warranty=None, incoterms=<Incoterms.DDP: 0>),
 ContractActual(price_per_unit=99.84999973695257, quantity=10485.728633000672, bundling_ind=None, payment_term=<PaymentTerm.NET60: 60>, delivery_timeline=None, contract_period=None, warranty=None, incoterms=<Incoterms.DDP: 0>))